# Predicting Brazilian Stock Market

## Reading the dataframe

In [1]:
from datetime import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr
import plotly.offline as pyo
import plotly.graph_objects as gp
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import numpy as np

pyo.init_notebook_mode(connected=True)

In [2]:
BRstocks = pd.read_csv('C:/Users/pablo/Alura/Portifolio/BRstocks.csv')
BRstocks.index = BRstocks['Date']
BRstocks = BRstocks.drop(['Date'], axis=1)
BRstocks

,High,Low,Open,Close,Volume,Stock
Date,,,,,,
2016-10-28,19.760000,18.920000,19.299999,19.200001,6342600.0,AALR3
2016-10-31,19.200001,17.510000,19.190001,18.059999,2523300.0,AALR3
2016-11-01,18.379999,17.160000,18.059999,17.900000,996200.0,AALR3
2016-11-03,18.240000,17.309999,18.000000,17.990000,621000.0,AALR3
2016-11-04,18.120001,17.709999,17.950001,17.750000,389800.0,AALR3
...,...,...,...,...,...,...
2021-11-29,21.400000,20.850000,21.200001,20.920000,1620800.0,YDUQ3
2021-11-30,22.280001,20.660000,21.400000,21.830000,4438500.0,YDUQ3
2021-12-01,23.040001,21.830000,21.959999,21.889999,3950300.0,YDUQ3


## Upadating the dataframe

### Selecting the time

In [3]:
end = dt.now()
start = BRstocks.index.max()
start = dt.strptime(start, '%Y-%m-%d')
start

datetime.datetime(2021, 12, 3, 0, 0)

In [4]:
stocks_names = pd.read_excel('../Dados/lala.xlsx')
s = stocks_names['Papel']
jorge = list(s.dropna())
names = []

for i in jorge:
    names.append(i + '.SA')

### Testing to see if any stock is missing

In [5]:
dale = pdr.get_data_yahoo(names, start, end)

In [6]:
dale

Attributes  Adj Close                                                 \
Symbols      AALR3.SA   ABCB4.SA ABEV3.SA ADHM3.SA AERI3.SA AESB3.SA   
Date                                                                   
2021-12-03  18.490000  17.200001    16.00     1.56     7.65    11.58   
2021-12-06  18.490000  17.680000    16.08     1.56     7.58    11.82   
2021-12-07  18.450001  17.549999    16.10     1.56     7.85    11.55   
2021-12-08  18.490000  17.910000    16.08     1.56     7.88    11.78   
2021-12-09  18.299999  17.799999    15.81     1.56     8.00    11.70   
2021-12-10  18.170000  17.910000    15.97     1.56     8.03    11.81   
2021-12-13  18.230000  17.730000    16.00      NaN     8.05    11.78   

Attributes                                         ...    Volume            \
Symbols     AFLT3.SA   AGRO3.SA AGXY3.SA AHEB3.SA  ...  VULC3.SA  VVEO3.SA   
Date                                               ...                       
2021-12-03  8.819558  24.010000     9.40     27.0  ...  439800.0  335400.0   
2021-12-06  8.830000  24.260000     9.49     27.0  ...  576900.0  156500.0   
2021-12-07  8.830000  24.290001     9.50     27.0  ...  592300.0   87000.0   
2021-12-08  8.950000  25.090000     9.50     27.0  ...  975100.0  339500.0   
2021-12-09  8.950000  24.639999     9.73     27.0  ...  790800.0  122300.0   
2021-12-10  8.950000  24.950001     9.75     27.0  ...  552700.0  574400.0   
2021-12-13  8.400000  25.680000     9.55      NaN  ...  412200.0   77800.0   

Attributes                                                               \
Symbols       WEGE3.SA   WEST3.SA WHRL3.SA WHRL4.SA   WIZS3.SA WLMM3.SA   
Date                                                                      
2021-12-03   8404300.0   940400.0   1500.0  13600.0  2528500.0      0.0   
2021-12-06   6809000.0   374300.0   3100.0  10200.0  2338700.0      0.0   
2021-12-07  12047300.0   263500.0   5500.0  11400.0  1482000.0      0.0   
2021-12-08  11021000.0  1172900.0   4600.0   4900.0  1665600.0      0.0   
2021-12-09  16902100.0   454100.0  10600.0   3800.0  1668000.0    100.0   
2021-12-10   5805900.0   394700.0   1400.0   2800.0  3230400.0      0.0   
2021-12-13   3145700.0   688500.0   1200.0   7100.0  2202600.0      NaN   

Attributes                      
Symbols    WLMM4.SA   YDUQ3.SA  
Date                            
2021-12-03    900.0  3705800.0  
2021-12-06   2100.0  2718900.0  
2021-12-07   1200.0  2497900.0  
2021-12-08    600.0  6492500.0  
2021-12-09   1100.0  1720300.0  
2021-12-10   1000.0  1651200.0  
2021-12-13    600.0  1061700.0  

[7 rows x 2832 columns]

### Actually updating the BRstocks

In [7]:
for i in names:
    x = pdr.get_data_yahoo(i, start, end)
    x['Stock'] = i[0:-3]
    x = x.drop(['Adj Close'], axis=1)
    BRstocks = BRstocks.append(x, False)

In [8]:
BRstocks

,High,Low,Open,Close,Volume,Stock
Date,,,,,,
2016-10-28,19.760000,18.920000,19.299999,19.200001,6342600.0,AALR3
2016-10-31,19.200001,17.510000,19.190001,18.059999,2523300.0,AALR3
2016-11-01,18.379999,17.160000,18.059999,17.900000,996200.0,AALR3
2016-11-03,18.240000,17.309999,18.000000,17.990000,621000.0,AALR3
2016-11-04,18.120001,17.709999,17.950001,17.750000,389800.0,AALR3
...,...,...,...,...,...,...
2021-12-07 00:00:00,24.740000,23.600000,24.629999,23.690001,2497900.0,YDUQ3
2021-12-08 00:00:00,24.639999,23.520000,23.959999,24.410000,6492500.0,YDUQ3
2021-12-09 00:00:00,24.320000,23.389999,23.980000,23.510000,1720300.0,YDUQ3


In [9]:
x

,High,Low,Open,Close,Volume,Stock
Date,,,,,,
2021-12-03,24.350000,23.129999,23.129999,23.760000,3705800,YDUQ3
2021-12-06,24.370001,23.370001,23.879999,23.830000,2718900,YDUQ3
2021-12-07,24.740000,23.600000,24.629999,23.690001,2497900,YDUQ3
2021-12-08,24.639999,23.520000,23.959999,24.410000,6492500,YDUQ3
2021-12-09,24.320000,23.389999,23.980000,23.510000,1720300,YDUQ3
2021-12-10,24.600000,23.750000,23.830000,24.040001,1651200,YDUQ3
2021-12-13,24.469999,23.639999,23.969999,23.930000,1083100,YDUQ3


In [10]:
BRstocks.index.max

<bound method IndexOpsMixin.max of Index([       '2016-10-28',        '2016-10-31',        '2016-11-01',
              '2016-11-03',        '2016-11-04',        '2016-11-07',
              '2016-11-08',        '2016-11-09',        '2016-11-10',
              '2016-11-11',
       ...
       2021-12-09 00:00:00, 2021-12-10 00:00:00, 2021-12-13 00:00:00,
       2021-12-03 00:00:00, 2021-12-06 00:00:00, 2021-12-07 00:00:00,
       2021-12-08 00:00:00, 2021-12-09 00:00:00, 2021-12-10 00:00:00,
       2021-12-13 00:00:00],
      dtype='object', name='Date', length=1667589)>

In [11]:
BRstocks.corr()

,High,Low,Open,Close,Volume
High,1.000000,0.999828,0.999860,0.999870,-0.001216
Low,0.999828,1.000000,0.999976,0.999974,-0.001213
Open,0.999860,0.999976,1.000000,0.999964,-0.001214
Close,0.999870,0.999974,0.999964,1.000000,-0.001214
Volume,-0.001216,-0.001213,-0.001214,-0.001214,1.000000


## Binary Classification prediction

### Using a sample

In [12]:
WEGE = BRstocks.loc[BRstocks['Stock'] == 'WEGE3']
WEGE = WEGE.drop(['Stock'], axis=1)
WEGE

,High,Low,Open,Close,Volume
Date,,,,,
2000-01-03,0.170118,0.168639,0.168639,0.168639,378560.0
2000-01-04,0.164201,0.162721,0.164201,0.162721,47320.0
2000-01-05,0.162721,0.162721,0.162721,0.162721,0.0
2000-01-06,0.147928,0.147928,0.147928,0.147928,250120.0
2000-01-07,0.149408,0.147928,0.149408,0.147928,135200.0
...,...,...,...,...,...
2021-12-07 00:00:00,34.560001,33.250000,33.320000,34.560001,12047300.0
2021-12-08 00:00:00,36.110001,34.209999,34.680000,36.040001,11021000.0
2021-12-09 00:00:00,37.110001,35.330002,35.750000,36.500000,16902100.0


In [13]:
WEGE['Close2'] = WEGE.Close.shift(periods=1) - WEGE.Close
WEGE

,High,Low,Open,Close,Volume,Close2
Date,,,,,,
2000-01-03,0.170118,0.168639,0.168639,0.168639,378560.0,NaN
2000-01-04,0.164201,0.162721,0.164201,0.162721,47320.0,0.005918
2000-01-05,0.162721,0.162721,0.162721,0.162721,0.0,0.000000
2000-01-06,0.147928,0.147928,0.147928,0.147928,250120.0,0.014793
2000-01-07,0.149408,0.147928,0.149408,0.147928,135200.0,0.000000
...,...,...,...,...,...,...
2021-12-07 00:00:00,34.560001,33.250000,33.320000,34.560001,12047300.0,-1.550003
2021-12-08 00:00:00,36.110001,34.209999,34.680000,36.040001,11021000.0,-1.480000
2021-12-09 00:00:00,37.110001,35.330002,35.750000,36.500000,16902100.0,-0.459999


In [14]:
WEGE.loc[WEGE['Close2'] < 0, 'Binary_Close'] = 0
WEGE.loc[WEGE['Close2'] >= 0, 'Binary_Close'] = 1
WEGE = WEGE[1:]
WEGE

,High,Low,Open,Close,Volume,Close2,Binary_Close
Date,,,,,,,
2000-01-04,0.164201,0.162721,0.164201,0.162721,47320.0,0.005918,1.0
2000-01-05,0.162721,0.162721,0.162721,0.162721,0.0,0.000000,1.0
2000-01-06,0.147928,0.147928,0.147928,0.147928,250120.0,0.014793,1.0
2000-01-07,0.149408,0.147928,0.149408,0.147928,135200.0,0.000000,1.0
2000-01-10,0.147928,0.147928,0.147928,0.147928,0.0,0.000000,1.0
...,...,...,...,...,...,...,...
2021-12-07 00:00:00,34.560001,33.250000,33.320000,34.560001,12047300.0,-1.550003,0.0
2021-12-08 00:00:00,36.110001,34.209999,34.680000,36.040001,11021000.0,-1.480000,0.0
2021-12-09 00:00:00,37.110001,35.330002,35.750000,36.500000,16902100.0,-0.459999,0.0


In [15]:
x = WEGE[['High', 'Low', 'Open', 'Volume']]
y = WEGE['Binary_Close']
y = y.astype('int')

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

SEED = 20


treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, 
                                                        random_state = SEED,
                                                        test_size = 0.25,
                                                        stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = LinearSVC(dual=False)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

Treinaremos com 4126 elementos e testaremos com 1376 elementos
A acurácia foi 67.15%


### Trying in the intire database

In [17]:
data = BRstocks
data

,High,Low,Open,Close,Volume,Stock
Date,,,,,,
2016-10-28,19.760000,18.920000,19.299999,19.200001,6342600.0,AALR3
2016-10-31,19.200001,17.510000,19.190001,18.059999,2523300.0,AALR3
2016-11-01,18.379999,17.160000,18.059999,17.900000,996200.0,AALR3
2016-11-03,18.240000,17.309999,18.000000,17.990000,621000.0,AALR3
2016-11-04,18.120001,17.709999,17.950001,17.750000,389800.0,AALR3
...,...,...,...,...,...,...
2021-12-07 00:00:00,24.740000,23.600000,24.629999,23.690001,2497900.0,YDUQ3
2021-12-08 00:00:00,24.639999,23.520000,23.959999,24.410000,6492500.0,YDUQ3
2021-12-09 00:00:00,24.320000,23.389999,23.980000,23.510000,1720300.0,YDUQ3


In [18]:
total = pd.DataFrame()
grouped = data.groupby(data.Stock)
for s in data.Stock.unique():
    globals()[f"{s}"] = grouped.get_group(f"{s}")
    globals()[f"{s}"]['Close2'] = globals()[f"{s}"].Close.shift(periods=-1) - globals()[f"{s}"].Close
    globals()[f"{s}"]['FutureClose'] = globals()[f"{s}"].Close.shift(periods=-1)
    globals()[f"{s}"].loc[globals()[f"{s}"]['Close2'] < 0, 'Binary_Close'] = 0
    globals()[f"{s}"].loc[globals()[f"{s}"]['Close2'] >= 0, 'Binary_Close'] = 1
    globals()[f"{s}"] = globals()[f"{s}"][1:]
    total = total.append(globals()[f"{s}"], True)

C:\Users\pablo\AppData\Local\Temp/ipykernel_6456/1378760754.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\pablo\AppData\Local\Temp/ipykernel_6456/1378760754.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\pablo\anaconda3\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

In [19]:
total

,High,Low,Open,Close,Volume,Stock,Close2,FutureClose,Binary_Close
0,19.200001,17.510000,19.190001,18.059999,2523300.0,AALR3,-0.16,17.90,0.0
1,18.379999,17.160000,18.059999,17.900000,996200.0,AALR3,0.09,17.99,1.0
2,18.240000,17.309999,18.000000,17.990000,621000.0,AALR3,-0.24,17.75,0.0
3,18.120001,17.709999,17.950001,17.750000,389800.0,AALR3,-0.10,17.65,0.0
4,18.000000,17.410000,17.980000,17.650000,513700.0,AALR3,-0.16,17.49,0.0
...,...,...,...,...,...,...,...,...,...
1667063,3.720000,3.450000,3.560000,3.450000,263500.0,WEST3,0.33,3.78,1.0
1667064,3.810000,3.390000,3.440000,3.780000,1172900.0,WEST3,-0.10,3.68,0.0
1667065,3.770000,3.540000,3.770000,3.680000,454100.0,WEST3,0.04,3.72,1.0
1667066,3.810000,3.650000,3.720000,3.720000,394700.0,WEST3,-0.08,3.64,0.0


In [20]:
total.corr()

,High,Low,Open,Close,Volume,Close2,FutureClose,Binary_Close
High,1.000000,0.999831,0.999860,0.999873,-0.001216,-0.060825,0.993158,0.031986
Low,0.999831,1.000000,0.999979,0.999974,-0.001213,-0.060667,0.993277,0.032002
Open,0.999860,0.999979,1.000000,0.999967,-0.001214,-0.060761,0.993260,0.031981
Close,0.999873,0.999974,0.999967,1.000000,-0.001214,-0.060809,0.993287,0.031975
Volume,-0.001216,-0.001213,-0.001214,-0.001214,1.000000,0.000007,-0.001213,-0.005894
Close2,-0.060825,-0.060667,-0.060761,-0.060809,0.000007,1.000000,0.055061,0.010892
FutureClose,0.993158,0.993277,0.993260,0.993287,-0.001213,0.055061,1.000000,0.033248
Binary_Close,0.031986,0.032002,0.031981,0.031975,-0.005894,0.010892,0.033248,1.000000


In [21]:
total = total.dropna()

In [22]:
x = total[['High', 'Low', 'Open', 'Volume', 'Close']]
y = total['Binary_Close']
y = y.astype('int')

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

SEED = 20


treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, 
                                                        random_state = SEED,
                                                        test_size = 0.25,
                                                        stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = LinearSVC(dual=False)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

Treinaremos com 1249911 elementos e testaremos com 416637 elementos
A acurácia foi 72.85%


### Simulator

In [24]:
def predictClassification(Ticket):
    
    Ticket = Ticket.dropna()
    
    x = Ticket[['High', 'Low', 'Open', 'Volume', 'Close']]
    y = Ticket['FutureClose']
    y = y.astype('int')
    
    SEED = 20
    
    treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, random_state = SEED, test_size = 0.25, stratify = y)
    print("Training with %d elements and Testing with %d elements" % (len(treino_x)*6, len(teste_x)*6))

    classification = LinearSVC(dual=False)
    classification.fit(treino_x, treino_y)
    previsoes = classification.predict(teste_x)

    acuracia = accuracy_score(teste_y, previsoes) * 100
    print("The accuracy score is %.2f%%" % acuracia)

    High = Ticket["High"].iloc[-1]
    Low = Ticket["Low"].iloc[-1]
    Open = Ticket["Open"].iloc[-1]
    Volume = Ticket["Volume"].iloc[-1]
    Close = Ticket["Close"].iloc[-1]
    entrada=[[High, Low, Open, Volume, Close]]
    
    if classification.predict(entrada) == [1]:
        print('This stock will increase or no change')
    elif classification.predict(entrada) == [0]:
        print('This stock will decrease')

In [25]:
predictClassification(MGLU3)

Training with 11790 elements and Testing with 3936 elements
The accuracy score is 57.16%
This stock will decrease


### Using Dummies

In [26]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier(strategy='stratified')
dummy_stratified.fit(treino_x, treino_y)
previsoes = dummy_stratified.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia do dummy stratified %.2f%%" % acuracia)

A acurácia do dummy stratified 60.36%


In [27]:
dummy_mostfrequent = DummyClassifier(strategy='most_frequent')
dummy_mostfrequent.fit(treino_x, treino_y)
acuracia = dummy_mostfrequent.score(teste_x, teste_y) * 100

print("A acurácia do dummy most frequent %.2f%%" % acuracia)

A acurácia do dummy most frequent 72.85%


In [28]:
dummy_mostfrequent = DummyClassifier(strategy='uniform')
dummy_mostfrequent.fit(treino_x, treino_y)
acuracia = dummy_mostfrequent.score(teste_x, teste_y) * 100

print("A acurácia do dummy uniform %.2f%%" % acuracia)

A acurácia do dummy uniform 50.07%


### Using the model

In [29]:
def predict(k, w, x, y, z):
    d = {'High': k, 'Low': w, 'Open': x, 'Volume': y, 'Close': z}
    df = pd.DataFrame(data=d)
    modelo.predict(df)
    if modelo.predict(df) == [1]:
        print('This stock will increase or no change')
    elif modelo.predict(df) == [0]:
        print('This stock will decrease')

In [30]:
predict([670], [670], [670], [4], [670])

This stock will increase or no change


## SGD Classifier

In [31]:
import numpy as np
from sklearn.linear_model import SGDClassifier

SEED = 20

treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, random_state = SEED, test_size = 0.25, stratify=y)

SGD = SGDClassifier(max_iter=1000, tol=0.01)
SGD.fit(treino_x, treino_y)
prediction = SGD.predict(teste_x)
score = SGD.score(treino_x, treino_y)

print('A Acurácia da previsão é de = {:.2f}%'.format(SGD.score(treino_x, treino_y)*100))

A Acurácia da previsão é de = 59.46%


In [32]:
    High = MGLU3["High"].iloc[-1]
    Low = MGLU3["Low"].iloc[-1]
    Open = MGLU3["Open"].iloc[-1]
    Volume = MGLU3["Volume"].iloc[-1]
    Close = MGLU3["Close"].iloc[-1]
    entrada=[[High, Low, Open, Volume, Close]]
    
    print('the Stock tomorrow will close at R$ {}'.format(SGD.predict(entrada)[0]))

the Stock tomorrow will close at R$ 1


## Linear Regression

In [33]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics

### Simulator

In [34]:
MGLU3

,High,Low,Open,Close,Volume,Stock,Close2,FutureClose,Binary_Close
Date,,,,,,,,,
2011-05-03,0.521562,0.50625,0.515625,0.509375,33670400.0,MGLU3,0.005625,0.515000,1.0
2011-05-04,0.515000,0.51000,0.510000,0.515000,39203200.0,MGLU3,-0.003125,0.511875,0.0
2011-05-05,0.512812,0.51000,0.512812,0.511875,35097600.0,MGLU3,-0.003750,0.508125,0.0
2011-05-06,0.511875,0.50000,0.510937,0.508125,38672000.0,MGLU3,-0.005000,0.503125,0.0
2011-05-09,0.507812,0.50125,0.506562,0.503125,667680.0,MGLU3,0.000000,0.503125,1.0
...,...,...,...,...,...,...,...,...,...
2021-12-07 00:00:00,7.720000,7.37000,7.490000,7.620000,85258900.0,MGLU3,-0.810000,6.810000,0.0
2021-12-08 00:00:00,7.570000,6.63000,7.560000,6.810000,255628900.0,MGLU3,-0.530000,6.280000,0.0
2021-12-09 00:00:00,6.760000,6.21000,6.760000,6.280000,257831300.0,MGLU3,0.090000,6.370000,1.0


In [35]:
def predictRegression(Ticket):
    
    Ticket = Ticket.dropna()
    
    x = Ticket[['High', 'Low', 'Open', 'Volume', 'Close']]
    y = Ticket['FutureClose']
    y = y.astype('int')
    
    SEED = 20
    
    treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, random_state = SEED, test_size = 0.25)
    print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x)*6, len(teste_x)*6))

    regression = LinearRegression()
    regression.fit(treino_x, treino_y)
    y_predicted = regression.predict(teste_x)

    print('A Acurácia da previsão é de = {:.2f}%'.format(metrics.r2_score(teste_y, y_predicted)*100))
    print('A Acurácia da previsão é de = {:.2f}%'.format(regression.score(treino_x, treino_y)*100))


    High = Ticket["High"].iloc[-1]
    Low = Ticket["Low"].iloc[-1]
    Open = Ticket["Open"].iloc[-1]
    Volume = Ticket["Volume"].iloc[-1]
    Close = Ticket["Close"].iloc[-1]
    entrada=[[High, Low, Open, Volume, Close]]
    
    print('the Stock tomorrow will close at R$ {}'.format(regression.predict(entrada)[0]))

In [36]:
predictRegression(MGLU3)

Treinaremos com 11790 elementos e testaremos com 3936 elementos
A Acurácia da previsão é de = 99.82%
A Acurácia da previsão é de = 99.75%
the Stock tomorrow will close at R$ 5.925378517364563
